In [73]:
import numpy as np
from scipy.linalg import eigvals
import pandas as pd


In [74]:
def calculate_cagr(initial_value, final_value, weeks_held):
    # Convert weeks to years
    years_held = weeks_held / 52
    # Calculate CAGR
    cagr = (final_value / initial_value) ** (1 / years_held) - 1
    return cagr

In [75]:
def yield_function(params):
    w2_slow, w1_medium = params
    w2_slow = int(w2_slow)
    w1_medium = int(w1_medium)
    stock = 'aapl'
    df = pd.read_csv(f'data/{stock}_quote_data.csv')
    df['slow'] = df['close'].rolling(w2_slow).mean()
    df['medium'] = df['close'].rolling(w1_medium).mean()
    df = df.dropna()

    close = df['close'].to_numpy()
    slow = df['slow'].to_numpy()
    medium = df['medium'].to_numpy()

    it = np.nditer(slow, flags=['f_index'])

    bought = False
    cost = 0
    cost_week = 0
    total_yield = 1
    total_weeks = 0

    for k in it:
        i = it.index
        slow_v = slow[i]
        medium_v = medium[i]
        close_v = close[i]
        if (not bought):
            if (medium_v > slow_v):
                bought = True
                cost = close_v
                cost_week = i
        if (bought):
            if (medium_v < slow_v):
                bought = False
                total_yield += total_yield*((close_v/cost)-1)
                total_weeks += (i-cost_week)
    final_cagr = calculate_cagr(1, total_yield, total_weeks) * 100
    return -final_cagr

In [76]:
def numerical_hessian(f, x, epsilon=1):
    n = len(x)
    hessian = np.zeros((n, n))
    perturb = np.eye(n) * epsilon

    for i in range(n):
        for j in range(n):
            f_x = f(x)
            f_x_i = f(x + perturb[i])
            f_x_j = f(x + perturb[j])
            f_x_ij = f(x + perturb[i] + perturb[j])
            hessian[i, j] = (f_x_ij - f_x_i - f_x_j + f_x) / (epsilon ** 2)
    return hessian


def is_positive_semi_definite(matrix):
    eigenvalues = eigvals(matrix)
    return np.all(eigenvalues >= 0)


# Define a grid of points over which to test convexity
w1_medium = np.linspace(start=20, stop=35, num=16, dtype=int)  # medium
w2_values = np.linspace(start=40, stop=60, num=21, dtype=int)  # slow
points = np.array(np.meshgrid(w1_medium, w2_values)).T.reshape(-1, 2)

psd = 0
npsd = 0
# Check Hessian at each point
for point in points:
    hessian = numerical_hessian(yield_function, point)
    if is_positive_semi_definite(hessian):
        psd += 1
        # print(f"At point {point}, the Hessian is positive semi-definite.")
    else:
        npsd += 1
        # print(f"At point {point}, the Hessian is NOT positive semi-definite.")
print('psd:', psd, 'npsd', npsd)

psd: 47 npsd 289
